## K-Means Clustering in Python

Source: http://stamfordresearch.com/k-means-clustering-in-python/ 

Source: https://www.youtube.com/watch?v=Lm1c2U8BmoA (PySpark, ML)

In [ ]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#set Matplotlib inline plotting and load Pandas package
%matplotlib inline
import pandas as pd
pd.options.display.mpl_style = 'default'

In [ ]:
# Load data 

data = hive_ctx.sql("Select * from bi_temp_kmeanClusteringtable")

In [ ]:
# Look at the first 5 results

df = data.toPandas()

df.head(5).transpose()

In [ ]:
# Number of features

len(df.columns)

### Summary Statistics

In [ ]:
df.describe().transpose().tail(5)

### Replace Nas by mean of column for variable x


In [ ]:
_ = df['x'].fillna(df['x'].mean(), inplace = True)

### Replace Nas by 0 for other columns

In [ ]:
_ = df.fillna(0, inplace = True)

In [ ]:
df.head(5).transpose()

In [ ]:
# Keep only 

cols = df.columns[(df.dtypes == 'int64') | (df.dtypes == 'float64')] # '|' = or 

len(cols)

In [ ]:
df[cols].head(5).transpose()

In [ ]:
# New DataFrame without the s__uid 

df = df[cols]

In [ ]:
# Matrix of covariance

sampled_data = df[cols].sample(frac=0.1) 

axs = pd.scatter_matrix(sampled_data, figsize=(12, 12)); 

# Rotate axis labels and remove axis ticks
n = len(sampled_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

In [ ]:
# Other type of visuzalization

# Source: http://datascience.stackexchange.com/questions/10459/calculation-and-visualization-of-correlation-matrix-with-pandas

def correlation_matrix(df):
    import numpy as np
    from matplotlib import pyplot as plt
    from matplotlib import cm as cm

    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    cmap = cm.get_cmap('jet', 30)
    cax = ax1.imshow(df.corr(), interpolation="nearest", cmap=cmap)
    ax1.grid(True)
    plt.title('Feature Correlation')
    labels= df.columns # not sure
    ax1.set_xticklabels(labels,fontsize=6)
    ax1.set_yticklabels(labels,fontsize=6)
    # Add colorbar, make sure to specify tick locations to match desired ticklabels
    cbar = fig.colorbar(cax, ticks=[.25,.3,.35,.4,.45,.5,.55,.6,.65,.70,.75,.8,.85,.90,.95,1])
    plt.show()
    
correlation_matrix(sampled_data)

 ### Standardize Features
 
 source: http://stackoverflow.com/questions/12525722/normalize-data-in-pandas 
 
"In cluster analysis variables with large values contribute more to the distance calculations. Variables measured on different scales should be standardized prior to clustering, so that the solution is not driven by variables measured on larger scales." 

In [ ]:
# standardize the data attributes

from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df)
df_normalized = pd.DataFrame(np_scaled)

df_normalized.head(5).transpose()

In [ ]:
# Set column names back

df_normalized.columns = cols

df_normalized.head().transpose()

### Features Selection 

- RandomForrest
- Lasso
- PCA

## K-mean clustering using Spark ML 

In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

In [ ]:
df_normalized.dtypes

type(1)

In [ ]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

df_normalized = sqlContext.createDataFrame(df_normalized)

'''
vectorAssembler = VectorAssembler(inputCols= df_normalized.columns,
                                  outputCol="features")

df = vectorAssembler.transform(df_normalized)
'''

rdd = df_normalized.map(lambda data: Vectors.dense([float(c) for c in data]))

clusters = KMeans.train(rdd, 5, maxIterations=10, initializationMode="random") # 5 clusters

In [ ]:
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer


class RowIterator(TransformerMixin):
    """ Prepare dataframe for DictVectorizer """
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return (row for _, row in X.iterrows())


vectorizer = make_pipeline(RowIterator(), DictVectorizer())

In [ ]:
# Compute the sum of Squared Error:

def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [ ]:
# Within-cluster sum of squares

WSSE = (rdd.map(lambda point: error(point))
                   .reduce(lambda x,y: x+y))

print("Within Set Sum of Squared Error = " + str(WSSE))

In [ ]:
# Try with a range of number of clusters

for l in range(1,6):
    clusters = KMeans.train(rdd, l, maxIterations = 100, runs = 100, initializationMode = 'random')
    WSSSE = (rdd.map(lambda point: error(point))
                .reduce(lambda x,y: x+y))
    print("With " + str(l) + ' clusters: Within Set Sum of Squared Error =' + str(WSSE))

## K-mean clustering using sklearn

### Sources

MasterClass: https://github.com/Marie-de-Leseleuc/Python-Code/blob/master/exo%2B1%20(2).ipynb 

1. https://www.datascience.com/blog/introduction-to-k-means-clustering-algorithm-learn-data-science-tutorials 

2. https://datasciencelab.wordpress.com/2013/12/12/clustering-with-k-means-in-python/

3. http://mnemstudio.org/clustering-k-means-example-1.htm

4. https://www.dataquest.io/blog/k-means-clustering-us-senators/

### Method 1: K-mean

In [ ]:
# Source: http://stackoverflow.com/questions/28017091/will-pandas-dataframe-object-work-with-sklearn-kmeans-clustering 

from sklearn.cluster import KMeans

dataset = df_normalized

# Convert DataFrame to matrix
mat = dataset.as_matrix()

# Using sklearn
km = KMeans(n_clusters=5)
km.fit(mat)

# Get cluster assignment labels
labels = km.labels_

# Format results as a DataFrame
 # results = pd.DataFrame([dataset.index,labels]).T # return a df with the cluster corresponding to each index

results = pd.DataFrame(data=labels, columns=['cluster'], index=dataset.index) # better way

In [ ]:
results.head(5)

In [ ]:
# Add cluster number to each observation 

print(len(dataset), len(results))

dataset_f = dataset

dataset_f['cluster'] = results # Add cluster number to df

dataset_f['id'] = data.toPandas()['id'] # add s__uid to df

dataset_f[['cluster', 'id']].head(5) # produce cluster by player

In [ ]:
# Describe the clusters

label = dataset_f.cluster.unique()

for label in set(labels):
    print("Label:",label)
    print(dataset_f[dataset_f["cluster"]==label].describe())

In [ ]:
# Draw histograms

for label in set(labels):
    print("Label:",label)
    dataset_f[dataset_f["cluster"]==label].hist()

In [ ]:
# Compare the cluster per variable

dataset_f.groupby('cluster').mean()

In [ ]:
# Personal exercice: denormalize the data

dataset.drop('id', axis=1, inplace=True)

dataset.drop('cluster', axis=1, inplace=True)

le = min_max_scaler.inverse_transform(dataset_f) # unscale the data

le = pd.DataFrame(le)

le.columns = cols

result = pd.DataFrame(data=labels, columns=['cluster'], index=le.index) 

dataset_f = le

dataset_f['cluster'] = result # Add cluster number to df

dataset_f['id'] = data.toPandas()['id'] # add s__uid to df

dataset_f[['cluster', 'id']].head(5) # produce cluster by player

dataset_f.groupby('cluster').mean()

In [ ]:
dataset_f.cluster.value_counts()

### Method 2: K-mean ++

In [ ]:
# Source: http://stackoverflow.com/questions/34958994/how-to-use-scikit-kmeans-when-i-have-a-dataframe

import sklearn
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split

dataset = df_normalized

sample_df_train, sample_df_test = sklearn.cross_validation.train_test_split(dataset, train_size=0.6)

cluster = sklearn.cluster.KMeans(n_clusters=5, init='k-means++', n_init=10, max_iter=300, tol=0.0001, 
                                 precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=1)

cluster.fit(sample_df_train)

result = cluster.predict(sample_df_train)

results = cluster.predict(sample_df_test)

In [ ]:
# Not sure it works....

labels = cluster.labels_

result = pd.DataFrame([sample_df_train.index,labels]).T 

result.head(5)

In [ ]:
# Merge the dataframe and the cluster analysis result

# Create a unique identifier using the index
sample_df_train.reset_index(level=0, inplace=True) 

# create a list with the index for each observation
cluslist=list(sample_df_train['index'])   

# create a lit with the cluster label for each observation
labels=list(cluster.labels_)

# create a dictionnary with the two lists
newlist=dict(zip(cluslist, labels))

# create a dataframe from the dictionnary 
newclus= pd.DataFrame.from_dict(newlist, orient='index')

# create a column with the cluster labels
newclus.columns = ['cluster']

# Create a unique identifier using the index
newclus.reset_index(level=0, inplace=True)

# merge the dataframes
merged_train=pd.merge(sample_df_train, newclus, on='index')

merged_train.head(n=5)

In [ ]:
# Describe the clusters

Listlabels = merged_train.cluster.unique()

labels = [x for x in labels if str(x) != 'nan'] # get rid of Nan

for label in set(labels):
    print("Label:",label)
    print(merged_train[merged_train["cluster"]==label].describe())

In [ ]:
merged_train.cluster.unique()

In [ ]:
# Compare clusters per variable

merged_train.groupby('cluster').mean()

In [ ]:
# Histograms

labels = range(0,1) # keep only the first 

for label in set(labels):
    print("Label:",label)
    merged_train[merged_train["cluster"]==label].hist()

### Method 3: K-mean + PCA

In [ ]:
#Source: 
#https://www.coursera.org/learn/machine-learning-data-analysis/lecture/Ebb2M/running-a-k-means-cluster-analysis-in-python-pt-1

import numpy as np
import matplotlib.pylab as plt
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing

dataset = df_normalized.drop('y', 1) # Will use variable y to assess the clusters i.e. get rid of it here. 

# split data into train and test sets
clus_train, clus_test = train_test_split(dataset, test_size=.3, random_state=123)

# k-means cluster analysis for 1-9 clusters                                                           
from scipy.spatial.distance import cdist
clusters=range(1,10)
meandist=[] #store distance values from the cluster centroids

for k in clusters:
    model=KMeans(n_clusters=k)  # specify number of clusters to use for the analysis
    model.fit(clus_train)  # cluster analysis
    clusassign=model.predict(clus_train) # assign cluster number to each observation based on the cluster analysis 
                                           # (i.e. assigned to closer cluster)
    meandist.append(sum(np.min(cdist(clus_train, model.cluster_centers_, 'euclidean'), axis=1)) # computes the average of 
                                            # the sum of the distances between each observation and a cluster centroids
    / clus_train.shape[0]) # divide the sum of distances by the nb of observation in the data set

In [ ]:
"""
Plot average distance from observations from the cluster centroid
to use the Elbow Method to identify number of clusters to choose
"""

plt.plot(clusters, meandist) # cluster 1 to 9 vs. avg distance that has just been calculated
plt.xlabel('Number of clusters') 
plt.ylabel('Average distance')
plt.title('Selecting k with the Elbow Method')

This graph shows the decrease in the avg minimun distance of the observations from the cluster centroids for each of the cluster solutions.

We can see that the avg distance decreases as the number of clusters increases. Since the goal of cluster analysis is to minimize the distance between observations and their assinged clusters, you want to chose the fewest amount of clusters that provide the low avg distance.

What we're looking for in this plot is a bend in the elbow that kind of shows where the average distance value might be leveling off such that adding more clusters doesn't decrease the average distance as much. 

Very subjective. Need to further examine the cluster solutions to see whether they do not overlap, wheter the patterns of means on the clustering variables are unique and meaningful and wether there are signifiant differences between the clusters on the external validation variable. 

In [ ]:
# Interpret 5 cluster solution (rerun the cluster analysis, this time asking for 5 clusters)
model3=KMeans(n_clusters=5) # create object 3 which contains the result from the cluster analaysis with 5 clusters
model3.fit(clus_train)      # fit the model
clusassign=model3.predict(clus_train)  # create an object that has the cluster assignment based on the 5 clusters model

# plot clusters (try to see if the clusters overlap with each other in the p-dimension space)

# 42 variables i.e. 42 dimensions, which would be impossible to vizualise. So will use canonical discriminant analysis 
# which is a data reduction technique that creates a smaller number of variables that are linear combinaison of 
# the 42 clustering variables.

# Canonical variables are ordered by proportion of variance accounted for.
# Majority of variance is accounted (i.e.accounts for as much of the variability in the data as possible) by first few 
# canonical variables and those are the ones that we can plot. 
# variability: dispersion: the extent to which a distribution is stretched or squeezed.

from sklearn.decomposition import PCA 
pca_2 = PCA(2) # returns the two first canonical variable

plot_columns = pca_2.fit_transform(clus_train) 
    # create a matrix that include the 2 canonical var. estimated by the PCA
    
    # PCA_2.fit asks Python to fit the canonical discriminate analysis that we specified with the PCA command,
    # and the _transform applies the canonical discriminate analysis to the clus_train data set to calculate 
    # the canonical variables
    
plt.scatter(x=plot_columns[:,0], y=plot_columns[:,1], c=model3.labels_,) # plot first and second canonical variables 
                                                                         # (first column & second column from plot_column)
        #  c=model3.labels_ tells python to color code the points for each of the clusters
    
plt.xlabel('Canonical variable 1')
plt.ylabel('Canonical variable 2')
plt.title('Scatterplot of Canonical Variables for 5 Clusters')
plt.show()

# .transform() explaination: http://stackoverflow.com/questions/27517425/apply-vs-transform-on-a-group-object


A canonical variate is a new variable (variate) formed by making a linear combination of two or more variates (variables) from a data set.

3 clusters with a good separation. Possible correlation between two others. 4 clusters better?

In [ ]:
"""
BEGIN multiple steps to merge cluster assignment with clustering variables to examine
cluster variable means by cluster
"""

# Let's look at the pattern of means on the clustering variables for each cluster to see whether they are distinct and meaningful
# To do this, we have to link the cluster assigment variables back to its corresponding observation in the clus_train dataset.

# create a unique identifier variable from the index for the 
# cluster training data to merge with the cluster assignment variable
clus_train.reset_index(level=0, inplace=True) # use the index to create a new variable labeled 'index' 
                                              # that we can use as a unique identifier.
  
    # level=0 tells Python to only remove the given levels from the index
    # inplace=True, tells Python to add the new column to the existing clus_train dataset.

# create a list that has the new index variable
cluslist=list(clus_train['index'])   

# create a list of cluster assignments 
labels=list(model3.labels_)

    # This will be combined with the cluster assignment variable,
    # so that we can merge the two datasets together by each observation's unique identifier.

In [ ]:
# combine index variable list with cluster assignment list into a dictionary
newlist=dict(zip(cluslist, labels)) # zip is used to combided list 
newlist

In [ ]:
# Select specific rows  
interesting_keys = range(0,6)
subdict = {x: newlist[x] for x in interesting_keys if x in newlist}
subdict

In [ ]:
# convert newlist dictionary to a dataframe
newclus= pd.DataFrame.from_dict(newlist, orient='index')
newclus.head(5)

In [ ]:
# rename the cluster assignment column
newclus.columns = ['cluster']

In [ ]:
newclus.head(5)

In [ ]:
# now do the same for the cluster assignment variable
# create a unique identifier variable from the index for the 
# cluster assignment dataframe 
# to merge with cluster training data

newclus.reset_index(level=0, inplace=True)

# merge the cluster assignment dataframe with the cluster training variable dataframe
# by the index variable

merged_train=pd.merge(clus_train, newclus, on='index')
merged_train.head(n=3)

In [ ]:
# cluster frequencies
merged_train.cluster.value_counts()

In [ ]:
"""
END multiple steps to merge cluster assignment with clustering variables to examine
cluster variable means by cluster
"""

In [ ]:
# FINALLY calculate clustering variable means by cluster
clustergrp = merged_train.groupby('cluster').mean() 
print ("Clustering variable means by cluster")
print(clustergrp)

Remember that the variables are standardise to be on the same scale with an overall sample mean of zero and a standard deviation of 1

Cluster 3, has the highest likelihood of connections to the game (and generaly of most variables). 

#### Let's see how the clusters differ on total playtime in the game: 

In [ ]:
# validate clusters in training data by examining cluster differences in y using ANOVA
# first have to merge y with clustering variables and cluster assignment data 

# Extract data from original dataset
tpt_data=df['y']

# split totalplaytime data into train and test sets
tpt_train, tpt_test = train_test_split(tpt_data, test_size=.3, random_state=123)

tpt_train1=pd.DataFrame(tpt_train)

tpt_train1.head(5)

In [ ]:
# merge the training observations in the dataset that includes the cluster assignment variable 
# with the training observations in the dataset that includes the clustering variables. 

tpt_train1.reset_index(level=0, inplace=True)

merged_train_all=pd.merge(tpt_train1, merged_train, on='index')

merged_train_all.head(5) 

In [ ]:
# Merge total play time and cluster assignment data

sub1 = merged_train_all[['y', 'cluster']].dropna()

sub1.head(10)

In [ ]:
# Is there significant differences between clusters on the quantitative totalplaytime variable?

import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 

# We use the ols function to test the analysis of variance. 
# The formula specifies the model, with y as the response variable and cluster, as the explanatory variable.

#The capital C tells Python that the cluster assignment variable is categorical

tptmod = smf.ols(formula='y ~ C(cluster)', data=sub1).fit()
print (tptmod.summary())

In [ ]:
# We can also print the mean y for each cluster using the groupby function

print ('means for y by cluster')
m1= sub1.groupby('cluster').mean()
print (m1)

In [ ]:
# And the standard deviation

print ('standard deviations for y by cluster')
m2= sub1.groupby('cluster').std()
print (m2)

In [ ]:
# Then, because our categorical cluster variable has 5 categories, 
# we will request a tukey test to evaluate post hot comparisons between the clusters 

mc1 = multi.MultiComparison(sub1['y'], sub1['cluster'])
res1 = mc1.tukeyhsd()
print(res1.summary())

The analysis of variance summary table indicates that the clusters do not differ significantly on total play time

### Method 4: Masterclass

In [ ]:
# Source: http://stackoverflow.com/questions/34175462/dendrogram-using-pandas-and-scipy

from scipy.cluster import hierarchy as hc

dataframe = merged_train
corr = 1 - dataframe.corr() 

corr_condensed = hc.distance.squareform(corr) # convert to condensed
z = hc.linkage(corr_condensed, method='average')
dendrogram = hc.dendrogram(z, labels=corr.columns)
plt.show()

Source: https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/

- horizontal lines are cluster merges
- vertical lines tell you which clusters/labels were part of merge forming that new cluster
- heights of the horizontal lines tell you about the distance that needed to be "bridged" to form the new cluster


In [ ]:
# Hierarchical Clustering (see Benjamin's document)

# Note: too much data? Seem to return an overflowing error

from scipy.cluster.hierarchy import dendrogram, linkage

Z = linkage(merged_train.as_matrix(), 'ward') 

# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()